* Load the gpt3 and the fine-tuned gpt2

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split, RandomSampler, SequentialSampler
torch.manual_seed(42)

from transformers import GPT2LMHeadModel,  GPT2Tokenizer, GPT2Config
from transformers import AdamW, get_linear_schedule_with_warmup

import openai

In [ ]:
# Load fine-tuned gpt2

# *************************
domain= 'Tabletop'
step=  28000

# domain= 'Kitchen'
# step=  276000
# *************************

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

model_path = './finetuned_small_lms/{}/gpt2_medium({})'.format(domain, step)
gpt2_m = GPT2LMHeadModel.from_pretrained(model_path).to(device)

model_path = './finetuned_small_lms/{}/gpt2_base({})'.format(domain, step)
gpt2_s = GPT2LMHeadModel.from_pretrained(model_path).to(device)


tokenizer_m = GPT2Tokenizer.from_pretrained('gpt2-medium', \
                                          bos_token='<|sos|>', \
                                          eos_token='<|eos|>', \
                                          pad_token='<|pad|>')

tokenizer_s = GPT2Tokenizer.from_pretrained('gpt2', \
                                          bos_token='<|sos|>', \
                                          eos_token='<|eos|>', \
                                          pad_token='<|pad|>')


In [ ]:
# Load gpt3.5 (refering to socratic model colab)
def GPT3(prompt, gpt_model="gpt-3.5-turbo-1106", max_tokens=3500, temperature=0, stop=None):
    
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages
    )
    
    return response['choices'][0]['message']['content'].strip()

In [ ]:
def GPT4(prompt, gpt_model="gpt-4-0613", max_tokens=3500, temperature=0, stop=None):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=gpt_model,
        messages=messages
    )
    
    return response['choices'][0]['message']['content'].strip()

In [ ]:
openai_api_key = # input your openai key
openai.api_key = openai_api_key # pip install openai==0.28

-----
* Load and check the sample commands file (type= json) and prompt file (type= txt)

In [ ]:
####### prompt file (input to gpt3.5)
if domain=='Tabletop':
    file_path=  './prompt_templates/fixed_objects/Action_steps_generating_prompt.txt'
else:
    file_path=  './prompt_templates/flexible_objects/Action_steps_generating_prompt.txt'

with open(file_path, 'r') as f:
    prompt_llm= f.read()

----
* (Test) Compare outputs between gpt3 and finetuned-gpt2

In [ ]:
sample_objects= ['Yellow semicircle block', 'Green square block', 'Red semicircle block', 'Yellow bowl', 'Red bowl']
sample_command=  "Arrange semicircle blocks in the same colored bowl as the picked block."
sample_prompt= "<|sos|>Objects= {}\nCommand= {}\nAction steps= \n".format(sample_objects, sample_command)

print(sample_prompt)

In [ ]:
####### gpt2-medium
gpt2_input= sample_prompt

### fine-tuned gpt2
gpt2_m.eval()
prompt_ids = torch.tensor(tokenizer_m.encode(gpt2_input)).unsqueeze(0).to(device)

gpt2_outputs = gpt2_m.generate(prompt_ids, 
#                                 do_sample=True, 
#                                 top_k=20, 
                                max_length = 1024,
#                                 top_p=0.7, 
                                pad_token_id=tokenizer_m.pad_token_id,
                                eos_token_id=tokenizer_m.eos_token_id,
#                                 num_return_sequences=1
                                )

for i, sample_output in enumerate(gpt2_outputs):
    output= tokenizer_m.decode(sample_output, skip_special_tokens=True)
    print(output)

In [ ]:
####### gpt2-small
gpt2_input= sample_prompt

### fine-tuned gpt2
gpt2_s.eval()
prompt_ids = torch.tensor(tokenizer_s.encode(gpt2_input)).unsqueeze(0).to(device)

gpt2_outputs = gpt2_s.generate(prompt_ids, 
#                                 do_sample=True, 
#                                 top_k=20, 
                                max_length = 1024,
#                                 top_p=0.7, 
                                pad_token_id=tokenizer_s.pad_token_id,
                                eos_token_id=tokenizer_s.eos_token_id,
#                                 num_return_sequences=1
                                )

for i, sample_output in enumerate(gpt2_outputs):
    output= tokenizer_s.decode(sample_output, skip_special_tokens=True)
    print(output)  

In [ ]:
####### gpt3.5 input
gpt3_input= prompt_llm.format(sample_objects, sample_command)

### gpt3.5
gpt3_output= GPT3(gpt3_input)

print('Objects=', sample_objects)
print('Command= ', sample_command)
print(gpt3_output)

In [ ]:
####### gpt4 input
gpt4_input= prompt_llm.format(sample_objects, sample_command)

### gpt4
gpt4_output= GPT4(gpt4_input)

print('Objects=', sample_objects)
print('Command= ', sample_command)
print(gpt4_output)